# Projeto final Programação inteira

#### Camila
#### Catalina
#### Sergio

In [1]:
import gurobipy as gp
from gurobipy import GRB

In [2]:
# Parameters 

# Transportation cost to Sao Paulo (SJ), Laranjeiras (LJ), Sao Bernardo (SB), Santo Andre (SA) e Sao Jose dos Campos (SJ)

cost = {'SP':{'A': 49,'B':48.9,'C1':49.92,'C2':117.83,'D':35}, 
        'LJ':{'A':56,'B':55.72,'C1':49.92,'C2':132.97},
        'SB':{'A':50,'B':49.31,'C1':49.92,'C2':117.83,'D':35},
        'SA':{'A':52,'B':55.08,'C1':49.92,'C2':117.83,'D':35},
        'SJ': {'A':46,'B':44,'C1':49.92,'C2':117.83}
        }

# Demanda de cada cidade

demand_SP = [4,5,4,3,4,5]
demand_LJ = [8,10,6,6,7,10]
demand_SB = [2,3,3,2,4,4]
demand_SA = [6,8,4,4,5,8]
demand_SJ = [2,1,2,1,3,3]

cities = list(cost.keys())

In [3]:
# Model for month 1

month = 1

model = gp.Model('Transportation cost')

x_SP = model.addVars(range(1,len(cost['SP'])+1),vtype=gp.GRB.INTEGER, name="x_SP")
x_LJ = model.addVars(range(1,len(cost['LJ'])+1),vtype=gp.GRB.INTEGER, name="x_LJ")
x_SB = model.addVars(range(1,len(cost['SB'])+1),vtype=gp.GRB.INTEGER, name="x_SB")
x_SA = model.addVars(range(1,len(cost['SA'])+1),vtype=gp.GRB.INTEGER, name="x_SA")
x_SJ = model.addVars(range(1,len(cost['SJ'])+1),vtype=gp.GRB.INTEGER, name="x_SJ")



Restricted license - for non-production use only - expires 2022-01-13


In [4]:
# Each company can ship cap[j] boxes to city j.

cap = [10,10,10,10,10]

capacity_SP = model.addConstrs((x_SP[j] <= cap[0] for j in range(1,len(cost['SP'])+1)), name='capacity to SP')
capacity_LJ = model.addConstrs((x_LJ[j] <= cap[1] for j in range(1,len(cost['LJ'])+1)), name='capacity to LJ')
capacity_SB = model.addConstrs((x_SB[j] <= cap[2] for j in range(1,len(cost['SB'])+1)), name='capacity to SB')
capacity_SA = model.addConstrs((x_SA[j] <= cap[3] for j in range(1,len(cost['SA'])+1)), name='capacity to SA')
capacity_SJ = model.addConstrs((x_SJ[j] <= cap[4] for j in range(1,len(cost['SJ'])+1)), name='capacity to SJ')

In [5]:
# Satisfy demand

c_demand_SP = model.addConstr(x_SP.sum() == demand_SP[month-1])
c_demand_LJ = model.addConstr(x_LJ.sum() == demand_LJ[month-1])
c_demand_SB = model.addConstr(x_SB.sum() == demand_SB[month-1])
c_demand_SA = model.addConstr(x_SA.sum() == demand_SA[month-1])
c_demand_SJ = model.addConstr(x_SJ.sum() == demand_SJ[month-1])

In [6]:
# Minimize cost

model.setObjective(gp.quicksum(list(cost['SP'].values())[j-1]*x_SP[j] for j in range(1,len(cost['SP'])+1))
                   + gp.quicksum(list(cost['LJ'].values())[j-1]*x_LJ[j] for j in range(1,len(cost['LJ'])+1))
                   +gp.quicksum(list(cost['SB'].values())[j-1]*x_SB[j] for j in range(1,len(cost['SB'])+1))
                   + gp.quicksum(list(cost['SA'].values())[j-1]*x_SA[j] for j in range(1,len(cost['SA'])+1))
                   + gp.quicksum(list(cost['SJ'].values())[j-1]*x_SJ[j] for j in range(1,len(cost['SJ'])+1)), GRB.MINIMIZE)

In [7]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 28 rows, 23 columns and 46 nonzeros
Model fingerprint: 0x0de74398
Variable types: 0 continuous, 23 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 1e+01]
Found heuristic solution: objective 1095.2000000
Presolve removed 28 rows and 23 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.04 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 907.36 1095.2 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.073600000000e+02, best bound 9.073600000000e+02, gap 0.0000%


In [8]:
for v in model.getVars():
    if v.x > 1e-6:
        print(v.varName,'=',v.x,'. Enviar a ', v.varName[2:4], v.x, 'caixas pela empresa transportadora',list(cost[v.varName[2:4]].keys())[int(v.varName[-2])-1])

# Display optimal total matching score
print('Costo total de transporte no mes ',month, ':', model.objVal)

x_SP[5] = 4.0 . Enviar a  SP 4.0 caixas pela empresa transportadora D
x_LJ[3] = 8.0 . Enviar a  LJ 8.0 caixas pela empresa transportadora C1
x_SB[5] = 2.0 . Enviar a  SB 2.0 caixas pela empresa transportadora D
x_SA[5] = 6.0 . Enviar a  SA 6.0 caixas pela empresa transportadora D
x_SJ[2] = 2.0 . Enviar a  SJ 2.0 caixas pela empresa transportadora B
Costo total de transporte no mes  1 : 907.36
